In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()  # Load variables from .env file

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY environment variable is not set")

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_mistralai import ChatMistralAI
from langchain_openai import ChatOpenAI


def get_new_chain():
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """
    **Important Directive**:
    Ignore any prior knowledge or context you may have about SMT-LIB, UNSAT Fusion, or related topics. Only use the information and guidelines provided in this prompt to perform the task.

    ### Task Definition
    You are an SMT expert in **UNSAT Fusion** — a technique for combining two unsatisfiable SMT-LIB formulas. Your task is to produce a new **unsatisfiable** and **syntactically correct** SMT-LIB formula by fusing two input formulas ω1 and ω2.

    ### SMT-LIB Syntax Guidelines:
    **The generated output must strictly conform to the SMT-LIB format:
    1. Variable Declarations: Use declare-fun to declare variables with their types: (declare-fun <var> () <type>).
    2. Function Definitions: Use define-fun to define functions: (define-fun <name> (<arg1 type1> <arg2 type2> ...) <return-type> <body>).
    3. Declare **ALL functions** in **prefix notation.**
    4. Assertions: Use assert to specify constraints: (assert <constraint>).
    5. Boolean Expressions:
        • Logical AND: (and <expr1> <expr2> ...)
        • Logical OR: (or <expr1> <expr2> ...)
        • Logical NOT: (not <expr>)
    6. Arithmetic Operations: Use standard operators like +, -, *, /, and div for integers.
    7. Relational Operators: Use comparison operators like >, <, >=, <=, and = for numerical expressions.
    8. Equality and Disequality: Use = for equality and distinct for disequality: (distinct <expr1> <expr2> ...).
    9. Conditionals: Use ite for if-then-else expressions: (ite <condition> <then-branch> <else-branch>).
    10. Quantifiers: Use forall and exists for quantified expressions:
        • Universal: (forall ((x <type>) ...) <body>)
        • Existential: (exists ((x <type>) ...) <body>)
    11. Constants: Declare constants directly using their types:
        • Integers: 1, -3
        • Real numbers: 1.0, -2.5
        • Booleans: true, false
    12. Bit-vectors: Use (_ bv<value> <bit-width>) for bit-vector constants.
    13. Bitwise consistency: Ensure all bitwise operations and operands have the same bit-width.
    14. Parentheses: Ensure all parentheses are balanced and correctly nested.

    ### UNSAT Fusion Steps:
    1. **Disjoin the Two Formulas**:
        • Disjoin the two input formulas **ω1** and **ω2** into a single formula.
        • Include all variable declarations from both formulas.
        • Link all assertions with logical OR. This will result in a deep nested expression. This expression **MUST** be syntactically correct and follow all SMT-LIB syntax rules.
        • Store this as an intermediate result.

    2. Introduce a Fresh Variable `z` not present in either formula.

    3. **Fusion Function**:
        • Identify the free variables in both formulas.
        • Define a fusion function z = f(...,...) that takes one free variable from ω1 and one free variable from ω2 as its arguments.
        • If both formulas have only one free variable and it is the same, the fusion function will take only one argument, namely the free variable present in both formulas.
        • Before finalizing the formula, verify that the combined constraints on z from both formulas are unsatisfiable. If they conflict, adjust numeric bounds or constants to avoid contradiction while preserving the overall structure of the constraints. 
        
    4. **Substitute Variables**:
        • Derive each free variable from the fusion function, in form of an expression in terms of 'z' and the other free variable. Example: z = f(x, y) = x + y => x = z - y and y = z - x.
        • If there exists only one free variable in ω1 and ω2, derive this free variable from the fusion function in form of an expression in terms of 'z'. Example: z = f(x) = 2 * x => x = z / 2.
        • Substitute **the first occurance of the free variable** in **ω1** with the expression derived from the fusion function.
        • Substitute **the first occurance of the free variable** in **ω2** with the expression derived from the fusion function.
        • Add the fusion function and the derived expressions as declaration blocks in the final formula.
        
    5. **Return the Final Formula**:
        • **ONLY return the fused formula, if it is guaranteed to be unsatisfiable.**
        • The final formula **MUST** be syntactically correct and follow all SMT-LIB syntax rules.
        • Validate parentheses matching before returning the formula.
        • Make sure the declarations of variables and functions are consistent with their usage in the formulas.

    ### Points to Consider:
    1. **Only return the final fused formula from step 5, without any explanation, comments, or intermediate steps.**
    2. **DO NOT** add any new assert blocks or modify existing constraints; only substitute as specified.
    3. **DO NOT** declare any new variables which are not present in the original formulas, except for `z`.
    4. The output must be a **raw SMT-LIB formula**, without any additional formatting, string delimiters, or code block markers.
    5. **Normalize Bit-Widths**:
        • Ensure all bit-vector operations involve operands of the same bit-width.
    6. **Do not wrap the output with triple quotes (`'''smt` or `'''`).** The formula should be returned exactly as it would appear in an SMT solver.
    7. **Variable names must remain unchanged**, except for `z`.
    8. For all bit-wise operations, ensure **Bit-Width Consistency**.
    9. Before returning the final fused formula, count the number of opening '(' and closing ')' parentheses.
	    •	The total number of '(' must exactly match the total number of ')'.
	    •	If the counts do not match, restructure the formula to fix the mismatch before returning it.

    10. Ensure the produced formula is syntactically correct and follows all SMT-LIB syntax rules.

    Now, perform the described operation for these formulas paying close attention to the details mentioned above:
            """
            ),
            (
                "human",
                """
    ω1: {formula1}
    ω2: {formula2}
    """
        ),
    ]
    )
    
    # llm = ChatMistralAI(
    #     model="mistral-large-latest",
    #     temperature=0,
    #     max_retries=2,
    #     api_key="KM3PGpAYB10SjVRj5IH5mysWswYiMJ4N"
    # )


    llm = ChatOpenAI(
        model="gpt-4o",
        temperature=0,
        max_tokens=None,
        timeout=300,
        max_retries=4,
        api_key=OPENAI_API_KEY
    )
    
    chain = prompt | llm
    return chain

In [2]:
import os
import random
import shutil

def get_random_formulas(source_dir, target_dir):

    """
    Extracts 100 random .smt2 files from the source directory and copies them to the target directory.
    """

    # Check if the source directory exists
    if not os.path.exists(source_dir):
        print(f"Source directory {source_dir} does not exist.")
        return
    # Ensure the target directory exists
    os.makedirs(target_dir, exist_ok=True)

    for file_name in os.listdir(target_dir):
        file_path = os.path.join(target_dir, file_name)
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)  # Remove file or symbolic link
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)  # Remove directory and its contents


    # Get a list of all .smt2 files in the source directory
    smt2_files = [f for f in os.listdir(source_dir) if f.endswith('.smt2')]

    # Check if there are enough files to select from
    if len(smt2_files) < 100:
        print(f"Not enough .smt2 files in {source_dir}. Found only {len(smt2_files)} files.")
    else:
        # Randomly select 100 files
        selected_files = random.sample(smt2_files, 100)

        # Copy the selected files to the target directory
        for file_name in selected_files:
            source_path = os.path.join(source_dir, file_name)
            target_path = os.path.join(target_dir, file_name)
            shutil.copy(source_path, target_path)

        print(f"Copied 100 .smt2 files to {target_dir}.")
get_random_formulas('./semantic-fusion-seeds/QF_BV/unsat', './seeds_BV_unsat')

Copied 100 .smt2 files to ./seeds_BV_unsat.


In [3]:
import os
import random
from validate_SMT import validate_and_solve_smt
import csv

def fuse_formula_pairs(directory, chain):
    smt_files = [os.path.join(directory, f) for f in sorted(os.listdir(directory)) if f.endswith('.smt2')]

    if len(smt_files) < 2:
        print("Not enough SMT formulas to create pairs.")
        return

    random.shuffle(smt_files)

    # Ensure each formula exists in only one pair
    pairs = [(smt_files[i], smt_files[i + 1]) for i in range(0, len(smt_files) - 1, 2)]

    # If there is an odd number of files, pair the last file with the first file
    if len(smt_files) % 2 != 0:
        pairs.append((smt_files[-1], smt_files[0]))

    valid_formula_counter = 0
    satisfiable_formula_counter = 0
    unsatisfiable_formula_counter = 0
    error_formula_counter = 0

    csv_file = "fused_results_unsat.csv"
    with open(csv_file, mode="w", newline="") as csvfile:
        csv_writer = csv.writer(csvfile)
        # Write header
        csv_writer.writerow(["Formula Pair", "Error", "Satisfiability", "Validity"])

        for file1, file2 in pairs:
            # Read in formula files
            with open(file1, "r") as f1, open(file2, "r") as f2:
                formula1 = f1.read()
                formula2 = f2.read()

            print(f"Processing pair:\nFormula 1 (from {file1}):\n")
            print(f"Formula 2 (from {file2}):\n")

            # Fuse formulas
            response = chain.invoke({"formula1": formula1, "formula2": formula2}, timeout=120)

            # Display result
            print(f"Fusion Result:\n{response.text()}\n")

            # Validation result
            validation_result = validate_and_solve_smt(response.content)
           
            error = None
            satisfiability = None
            validity = None

            if validation_result.startswith("SMT formula is valid"):
                validity = "Valid"
                valid_formula_counter += 1
            if "Satisfiability result: sat" in validation_result:
                satisfiability = "SAT"
                satisfiable_formula_counter += 1
            elif "Satisfiability result: unsat" in validation_result:
                satisfiability = "UNSAT"
                unsatisfiable_formula_counter += 1
            if "Error" in validation_result or "error" in validation_result:
                error = validation_result
                error_formula_counter += 1

            # Write to CSV
            csv_writer.writerow([f"{file1} & {file2}", error, satisfiability, validity])

    print("\nValid Fused Formulas: " + f"{valid_formula_counter}\n")
    print("SAT Fused Formulas: " + f"{satisfiable_formula_counter}\n")
    print("UNSAT Fused Formulas: " + f"{unsatisfiable_formula_counter}\n")
    print("Fused Formulas with Errors: " + f"{error_formula_counter}\n")
    print(f"Results saved to {csv_file}")
fuse_formula_pairs("./seeds_BV_unsat", get_new_chain())

SMT formula is valid.
Satisfiability result: unsat
Processing pair:
Formula 1 (from ./seeds_BV_unsat/bench_6796.smt2):

Formula 2 (from ./seeds_BV_unsat/bench_8459.smt2):

Fusion Result:
(set-info :smt-lib-version 2.6)
(set-logic QF_BV)
(set-info :source |
 Patrice Godefroid, SAGE (systematic dynamic test generation)
 For more information: http://research.microsoft.com/en-us/um/people/pg/public_psfiles/ndss2008.pdf
|)
(set-info :category "industrial")
(set-info :status unsat)
(declare-fun T1_114127 () (_ BitVec 8))
(declare-fun T1_114128 () (_ BitVec 8))
(declare-fun T1_114129 () (_ BitVec 8))
(declare-fun T1_7 () (_ BitVec 8))
(declare-fun T1_5 () (_ BitVec 8))
(declare-fun z () (_ BitVec 8))
(define-fun fusion_function ((x (_ BitVec 8)) (y (_ BitVec 8))) (_ BitVec 8) (bvadd x y))
(assert (let ((?v_3 ((_ zero_extend 24) (_ bv1 8))) (?v_2 ((_ zero_extend 24) (_ bv2 8))) (?v_0 (bvsub ((_ zero_extend 24) T1_114127) (_ bv48 32)))) (let ((?v_1 (bvsub (bvadd ((_ zero_extend 24) T1_114128) (

Exception ignored in: <function Solver.__del__ at 0x113a9f6d0>
Traceback (most recent call last):
  File "/Users/dara/.local/lib/python3.10/site-packages/z3/z3.py", line 6881, in __del__
    Z3_solver_dec_ref(self.ctx.ref(), self.solver)
  File "/Users/dara/.local/lib/python3.10/site-packages/z3/z3core.py", line 3881, in Z3_solver_dec_ref
    _elems.Check(a0)
  File "/Users/dara/.local/lib/python3.10/site-packages/z3/z3core.py", line 1459, in Check
    raise self.Exception(self.get_error_message(ctx, err))
z3.z3types.Z3Exception: b'(error "line 14 column 0: invalid assert command, \')\' expected")\n'


Fusion Result:
(set-info :smt-lib-version 2.6)
(set-logic QF_BV)
(set-info :source |
 Patrice Godefroid, SAGE (systematic dynamic test generation)
 For more information: http://research.microsoft.com/en-us/um/people/pg/public_psfiles/ndss2008.pdf
|)
(set-info :category "industrial")
(set-info :status unsat)
(declare-fun T1_114493 () (_ BitVec 8))
(declare-fun T1_124523 () (_ BitVec 8))
(declare-fun z () (_ BitVec 8))
(define-fun f ((x (_ BitVec 8)) (y (_ BitVec 8))) (_ BitVec 8) (bvadd x y))
(assert (let ((?v_0 ((_ zero_extend 24) (bvsub z T1_124523)))) (or (and true (= (bvand (bvor (bvshl (bvand ?v_0 (_ bv255 32)) ((_ zero_extend 24) (_ bv6 8))) (_ bv28 32)) (_ bv127 32)) (_ bv4294967295 32)) (not (= ?v_0 (_ bv4294967295 32)))) (and true (= ((_ zero_extend 24) (bvsub z T1_114493)) (_ bv4294967295 32)))))
(check-sat)
(exit)

Processing pair:
Formula 1 (from ./seeds_BV_unsat/bench_4871.smt2):

Formula 2 (from ./seeds_BV_unsat/bench_8010.smt2):



Exception ignored in: <function Solver.__del__ at 0x113a9f6d0>
Traceback (most recent call last):
  File "/Users/dara/.local/lib/python3.10/site-packages/z3/z3.py", line 6881, in __del__
    Z3_solver_dec_ref(self.ctx.ref(), self.solver)
  File "/Users/dara/.local/lib/python3.10/site-packages/z3/z3core.py", line 3881, in Z3_solver_dec_ref
    _elems.Check(a0)
  File "/Users/dara/.local/lib/python3.10/site-packages/z3/z3core.py", line 1459, in Check
    raise self.Exception(self.get_error_message(ctx, err))
z3.z3types.Z3Exception: b'(error "line 30 column 2505: Sort mismatch at argument #3 for function (declare-fun and (Bool Bool) Bool) supplied sort is (_ BitVec 32)")\n'


Fusion Result:
(set-info :smt-lib-version 2.6)
(set-logic QF_BV)
(set-info :source |
 Patrice Godefroid, SAGE (systematic dynamic test generation)
 For more information: http://research.microsoft.com/en-us/um/people/pg/public_psfiles/ndss2008.pdf
|)
(set-info :category "industrial")
(set-info :status unsat)
(declare-fun T1_122043 () (_ BitVec 8))
(declare-fun T1_122044 () (_ BitVec 8))
(declare-fun T1_122045 () (_ BitVec 8))
(declare-fun T1_122046 () (_ BitVec 8))
(declare-fun T1_122047 () (_ BitVec 8))
(declare-fun T1_122048 () (_ BitVec 8))
(declare-fun T1_122049 () (_ BitVec 8))
(declare-fun T1_122050 () (_ BitVec 8))
(declare-fun T1_122051 () (_ BitVec 8))
(declare-fun T1_122052 () (_ BitVec 8))
(declare-fun T1_114101 () (_ BitVec 8))
(declare-fun T1_114102 () (_ BitVec 8))
(declare-fun T1_114103 () (_ BitVec 8))
(declare-fun T1_114104 () (_ BitVec 8))
(declare-fun T1_124 () (_ BitVec 8))
(declare-fun T1_122 () (_ BitVec 8))
(declare-fun T1_123 () (_ BitVec 8))
(declare-fun T1_126 

...

Fusion Result:
(set-info :smt-lib-version 2.6)
(set-logic QF_BV)
(set-info :source |
Patrice Godefroid, SAGE (systematic dynamic test generation)
For more information: http://research.microsoft.com/en-us/um/people/pg/public_psfiles/ndss2008.pdf
|)
(set-info :category "industrial")
(set-info :status unsat)
(declare-fun T2_613 () (_ BitVec 16))
(declare-fun T1_613 () (_ BitVec 8))
(declare-fun T1_614 () (_ BitVec 8))
(declare-fun T1_430 () (_ BitVec 8))
(declare-fun T1_431 () (_ BitVec 8))
(declare-fun T1_432 () (_ BitVec 8))
(declare-fun T1_433 () (_ BitVec 8))
(declare-fun T1_434 () (_ BitVec 8))
(declare-fun T1_435 () (_ BitVec 8))
(declare-fun T1_436 () (_ BitVec 8))
(declare-fun T1_437 () (_ BitVec 8))
(declare-fun T1_438 () (_ BitVec 8))
(declare-fun T1_439 () (_ BitVec 8))
(declare-fun z () (_ BitVec 8))
(define-fun fusion_function ((x (_ BitVec 8)) (y (_ BitVec 8))) (_ BitVec 8) (bvadd x y))
(assert (or (and true (= T2_613 (bvor (bvshl ((_ zero_extend 8) T1_614) (_ bv8 16)) ((_ z

Exception ignored in: <function Solver.__del__ at 0x113a9f6d0>
Traceback (most recent call last):
  File "/Users/dara/.local/lib/python3.10/site-packages/z3/z3.py", line 6881, in __del__
    Z3_solver_dec_ref(self.ctx.ref(), self.solver)
  File "/Users/dara/.local/lib/python3.10/site-packages/z3/z3core.py", line 3881, in Z3_solver_dec_ref
    _elems.Check(a0)
  File "/Users/dara/.local/lib/python3.10/site-packages/z3/z3core.py", line 1459, in Check
    raise self.Exception(self.get_error_message(ctx, err))
z3.z3types.Z3Exception: b'(error "line 18 column 2211: invalid command, \'(\' expected")\n'


Fusion Result:
(set-info :smt-lib-version 2.6)
(set-logic QF_BV)
(set-info :source |
 Patrice Godefroid, SAGE (systematic dynamic test generation)
 For more information: http://research.microsoft.com/en-us/um/people/pg/public_psfiles/ndss2008.pdf
|)
(set-info :category "industrial")
(set-info :status unsat)
(declare-fun T1_99272 () (_ BitVec 8))
(declare-fun T1_99279 () (_ BitVec 8))
(declare-fun T1_99282 () (_ BitVec 8))
(declare-fun T1_99287 () (_ BitVec 8))
(declare-fun T1_99296 () (_ BitVec 8))
(declare-fun T1_99306 () (_ BitVec 8))
(declare-fun T1_307 () (_ BitVec 8))
(declare-fun z () (_ BitVec 8))
(define-fun fusion_function ((x (_ BitVec 8)) (y (_ BitVec 8))) (_ BitVec 8) (bvadd x y))
(assert (let ((?v_8 (bvand (bvand ((_ zero_extend 24) T1_99272) (_ bv255 32)) (_ bv31 32)))) (let ((?v_10 (bvadd ?v_8 ((_ zero_extend 24) (_ bv1 8))))) (let ((?v_13 (bvadd ?v_10 (_ bv267 32))) (?v_0 ((_ zero_extend 24) (_ bv2 8)))) (let ((?v_4 (bvand (bvor (bvshl (bvand ((_ zero_extend 24) T1_9927

Exception ignored in: <function Solver.__del__ at 0x113a9f6d0>
Traceback (most recent call last):
  File "/Users/dara/.local/lib/python3.10/site-packages/z3/z3.py", line 6881, in __del__
    Z3_solver_dec_ref(self.ctx.ref(), self.solver)
  File "/Users/dara/.local/lib/python3.10/site-packages/z3/z3core.py", line 3881, in Z3_solver_dec_ref
    _elems.Check(a0)
  File "/Users/dara/.local/lib/python3.10/site-packages/z3/z3core.py", line 1459, in Check
    raise self.Exception(self.get_error_message(ctx, err))
z3.z3types.Z3Exception: b'(error "line 15 column 2523: invalid command, \'(\' expected")\n'


Fusion Result:
(set-info :smt-lib-version 2.6)
(set-logic QF_BV)
(set-info :source |
 Patrice Godefroid, SAGE (systematic dynamic test generation)
 For more information: http://research.microsoft.com/en-us/um/people/pg/public_psfiles/ndss2008.pdf
|)
(set-info :category "industrial")
(set-info :status unsat)
(declare-fun T1_114127 () (_ BitVec 8))
(declare-fun T1_114128 () (_ BitVec 8))
(declare-fun T1_114129 () (_ BitVec 8))
(declare-fun T1_10981 () (_ BitVec 8))
(declare-fun z () (_ BitVec 8))
(define-fun fusion_function ((x (_ BitVec 8)) (y (_ BitVec 8))) (_ BitVec 8) (bvadd x y))
(assert (let ((?v_3 ((_ zero_extend 24) (_ bv1 8))) (?v_9 ((_ zero_extend 24) (_ bv16 8))) (?v_4 ((_ zero_extend 24) (_ bv3 8))) (?v_2 ((_ zero_extend 24) (_ bv2 8))) (?v_0 (bvsub ((_ zero_extend 24) T1_114127) (_ bv48 32)))) (let ((?v_1 (bvsub (bvadd ((_ zero_extend 24) T1_114128) (bvshl (bvadd ?v_0 (bvshl ?v_0 ?v_2)) ?v_3)) (_ bv48 32)))) (let ((?v_5 (bvadd ((_ zero_extend 24) T1_114129) (bvshl (bvadd ?v_

In [ ]:
from validate_SMT import validate_and_solve_smt
formula1 = """
(set-info :smt-lib-version 2.6)
(set-logic QF_BV)
(set-info :source |
 Patrice Godefroid, SAGE (systematic dynamic test generation)
 For more information: http://research.microsoft.com/en-us/um/people/pg/public_psfiles/ndss2008.pdf
|)
(set-info :category "industrial")
(set-info :status unsat)
(declare-fun T1_111348 () (_ BitVec 8))
(assert (let ((?v_0 (bvand (bvand ((_ zero_extend 24) T1_111348) (_ bv255 32)) (_ bv31 32)))) (let ((?v_1 (bvadd ?v_0 ((_ zero_extend 24) (_ bv1 8))))) (let ((?v_2 (bvadd ?v_1 (_ bv267 32)))) (and true (bvsle ?v_2 (_ bv122 32)) (not (= ?v_0 (_ bv4294967295 32))) (bvsle ?v_1 (_ bv30 32)) (bvsle (_ bv121 32) ?v_2) (bvsle (_ bv81 32) ?v_2) (bvslt (_ bv0 32) ?v_2))))))
(check-sat)
(exit)

"""

formula2 = """
(set-info :smt-lib-version 2.6)
(set-logic QF_BV)
(set-info :source |
 Patrice Godefroid, SAGE (systematic dynamic test generation)
 For more information: http://research.microsoft.com/en-us/um/people/pg/public_psfiles/ndss2008.pdf
|)
(set-info :category "industrial")
(set-info :status unsat)
(declare-fun T1_111348 () (_ BitVec 8))
(assert (let ((?v_0 (bvand (bvand ((_ zero_extend 24) T1_111348) (_ bv255 32)) (_ bv31 32)))) (let ((?v_1 (bvadd ?v_0 ((_ zero_extend 24) (_ bv1 8))))) (let ((?v_2 (bvadd ?v_1 (_ bv267 32)))) (and true (bvslt ?v_2 (_ bv121 32)) (not (= ?v_0 (_ bv4294967295 32))) (bvsle ?v_1 (_ bv30 32)) (bvsle (_ bv81 32) ?v_2) (bvslt (_ bv0 32) ?v_2))))))
(check-sat)
(exit)

"""


response = get_new_chain().invoke({
    "formula1": formula1,
    "formula2": formula2
})

print(response.text() + '\n')
# print(validate_and_solve_smt(response.content))